In [47]:
import os
NEWSORG_API_KEY = os.getenv('NEWORG_API_KEY')
ALERT_DATA_PATH = "./firebase/alerts.csv"

In [2]:
from ollama import chat
from ollama import ChatResponse


response: ChatResponse = chat(model='llama3', messages=[
    {
        
        'role': 'user',
        'content': 'Only say †he work "Working"',
    },
])

print(response.message.content)

Working


In [48]:
import pandas as pd

# Read the CSV file into a DataFrame
alert_df = pd.read_csv(ALERT_DATA_PATH)

# Basic statistics
print(alert_df.describe())

         latitude   longitude  num_affirmatives  num_responses     cluster  \
count  250.000000  250.000000        250.000000     250.000000  250.000000   
mean    28.933467   35.861660         24.820000      34.968000   19.528000   
std      2.241661    9.274623         14.412024      15.658471   22.654229   
min     24.426816   31.200168          0.000000       2.000000    0.000000   
25%     30.039081   31.218825         13.000000      22.000000    1.000000   
50%     30.049077   31.242431         25.000000      34.500000   12.000000   
75%     30.057574   31.264042         37.000000      48.750000   29.000000   
max     30.064881   54.399346         50.000000      69.000000   82.000000   

       prob_true_random_forrest  
count                250.000000  
mean                   0.665564  
std                    0.159975  
min                    0.107096  
25%                    0.593105  
50%                    0.693515  
75%                    0.776237  
max                    0.

In [49]:
import ipywidgets as widgets
from IPython.display import display

# Display the first few rows as a table widget
output_widget = widgets.Output()
with output_widget:
    display(alert_df.head())

display(output_widget)

Output()

In [44]:
import requests
from time import sleep

# Function to fetch related news using a web search API
def get_related_news(query, num_results=5):
    search_url = f"https://newsapi.org/v2/everything?q={query}&sortBy=relevancy&apiKey={API_KEY}"
    
    try:
        response = requests.get(search_url)
        response.raise_for_status()
        news_data = response.json()

        if news_data.get("articles"):
            return [
                {"title": article["title"], "url": article["url"], "source": article["source"]["name"]}
                for article in news_data["articles"][:num_results]
            ]
        return []
    except requests.exceptions.RequestException as e:
        print(f"Error fetching news: {e}")
        return []

In [60]:
from IPython.display import display


# Iterate through rows and fetch related news
processed_alert_df = alert_df.apply(lambda row: row.to_dict(), axis=1)
results = {}
for row in processed_alert_df[:5]:
    # related_news = get_related_news(f"{row['title']} {row['location_description']}")
    related_news = get_related_news(f"{row['title']}")
    results[row['alert_id']] = related_news
    sleep(0.5)  # Avoid API rate limits

In [61]:
news_df = pd.DataFrame(results)
display(news_df)  # Displays a nicely formatted DataFrame in Jupyter

,d6114146-192e-410a-9513-2c9c71736832,da87c183-aef2-4ed2-8623-056b34d071fe,e364c45e-08a5-4d91-b8d3-1e0fc8f93696,cf55cf93-68f8-4eee-a4fb-fd6deacabd16,5152ca9b-d2fb-4b8e-ad38-bc24105e9258
0,{'title': 'Trumplican 6th Circuit Just Killed ...,{'title': 'How Disaster Provides Cover for Aut...,"{'title': 'Looters dressed as firefighters, wa...","{'title': 'Looters dressed as firefighters, wa...",{'title': 'Esperts found new DoNot Team APT gr...
1,"{'title': 'The most viewed articles of 2024', ...",{'title': 'New York Times Defends Mexican Cart...,{'title': 'Suspect arrested in the stabbing of...,{'title': 'Suspect arrested in the stabbing of...,{'title': 'Experts found multiple flaws in Mer...
2,{'title': 'New Orleans attacker had researched...,"{'title': 'Trump versus the Gulf of Mexico', '...",{'title': 'Actor Saif Ali Khan injured in knif...,{'title': 'Actor Saif Ali Khan injured in knif...,{'title': 'Critical remote code execution bug ...
3,"{'title': 'The Grift That Has Been Joe Biden',...",{'title': 'Illegal immigrant from Dominican Re...,{'title': 'Saif Ali Khan attacked: Minister de...,{'title': 'Saif Ali Khan attacked: Minister de...,{'title': 'MikroTik botnet relies on DNS misco...
4,{'title': 'Full NFL playoff picks with one sur...,{'title': 'Trump designates drug cartels as te...,{'title': 'Saif Ali Khan Stabbed: Star Attacke...,{'title': 'Saif Ali Khan Stabbed: Star Attacke...,{'title': 'DeepSeek database exposed highly se...
